In [ ]:
!pip install -q albumentations segmentation-models-pytorch timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.4 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)



In [ ]:
DATASET_PATH = '/content/drive/MyDrive/Dataset(main)'
IMAGES_PATH = os.path.join(DATASET_PATH, 'images')
MASKS_PATH = os.path.join(DATASET_PATH, 'masks')

print(f"DATASET_PATH: {DATASET_PATH}")
print(f"IMAGES_PATH: {IMAGES_PATH}")
print(f"MASKS_PATH: {MASKS_PATH}")

In [ ]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


In [ ]:
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.GaussianBlur(p=0.2),

    # 🔥 CRITICAL FIX
    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    ),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    ),
    ToTensorV2()
])


In [ ]:
class ArchaeologyDataset(Dataset):
    def __init__(self, image_dir, mask_dir, files, transforms=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.files = files
        self.transforms = transforms

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_name = self.files[idx]

        image = cv2.imread(os.path.join(self.image_dir, img_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(
            os.path.join(self.mask_dir, img_name),
            cv2.IMREAD_GRAYSCALE
        )

        if self.transforms:
            augmented = self.transforms(image=image, mask=mask)
            image = augmented["image"]
            mask = augmented["mask"]

        # ✅ FIX: convert dtype only
        mask = mask.long()

        return image, mask


In [ ]:
valid_exts = (".png", ".jpg", ".jpeg")

all_files = sorted([
    f for f in os.listdir(IMAGES_PATH)
    if f.lower().endswith(valid_exts)
])

train_files, val_files = train_test_split(
    all_files, test_size=0.2, random_state=42
)

train_ds = ArchaeologyDataset(IMAGES_PATH, MASKS_PATH, train_files, train_transform)
val_ds   = ArchaeologyDataset(IMAGES_PATH, MASKS_PATH, val_files, val_transform)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=4, shuffle=False)

print("Train:", len(train_ds), "Val:", len(val_ds))

In [ ]:
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=3
).to(device)


In [ ]:
!pip install -q albumentations segmentation-models-pytorch timm

In [ ]:
dice_loss  = smp.losses.DiceLoss(mode="multiclass")
focal_loss = smp.losses.FocalLoss(mode="multiclass")

def combined_loss(pred, target):
    return dice_loss(pred, target) + focal_loss(pred, target)


In [ ]:
def mean_iou(pred, target, num_classes=3):
    pred = pred.argmax(dim=1)
    ious = []

    for cls in range(num_classes):
        pred_c = pred == cls
        target_c = target == cls

        inter = (pred_c & target_c).sum().float()
        union = (pred_c | target_c).sum().float()

        if union == 0:
            ious.append(torch.tensor(1.0, device=device))
        else:
            ious.append(inter / union)

    return torch.mean(torch.stack(ious))


In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(), lr=1e-4, weight_decay=1e-5
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", factor=0.5, patience=3
)

scaler = torch.amp.GradScaler('cuda')

In [ ]:
EPOCHS = 50
best_iou = 0.0
accumulation_steps = 4  # Effective batch size = 16

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    optimizer.zero_grad()

    for i, (images, masks) in enumerate(tqdm(train_loader)):
        images, masks = images.to(device), masks.to(device)

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = combined_loss(outputs, masks)
            loss = loss / accumulation_steps

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        train_loss += loss.item() * accumulation_steps

    model.eval()
    val_iou = 0
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            val_iou += mean_iou(outputs, masks)

    val_iou /= len(val_loader)
    scheduler.step(val_iou)

    lr = optimizer.param_groups[0]["lr"]
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss/len(train_loader):.4f}")
    print(f"Val IoU: {val_iou:.4f} | LR: {lr:.2e}")

    if val_iou > best_iou:
        best_iou = val_iou
        torch.save(
            model.state_dict(),
            "/content/drive/MyDrive/best_unet_week3_fixed.pth"
        )
        print("✅ Model saved!")

In [ ]:
import torch
import segmentation_models_pytorch as smp

device = "cuda" if torch.cuda.is_available() else "cpu"

model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights=None,  # IMPORTANT for inference
    in_channels=3,
    classes=3
).to(device)

model.load_state_dict(
    torch.load("/content/drive/MyDrive/best_unet_week3_fixed.pth",
               map_location=device)
)

model.eval()
print("✅ Model loaded")


In [ ]:
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2

infer_transform = A.Compose([
    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    ),
    ToTensorV2()
])

def predict_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    aug = infer_transform(image=image)
    img_tensor = aug["image"].unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)
        pred_mask = torch.argmax(output, dim=1)

    return image, pred_mask.squeeze().cpu().numpy()


In [ ]:
import matplotlib.pyplot as plt

def visualize_prediction(idx):
    image_tensor, gt_mask_tensor = val_ds[idx]
    image_display = image_tensor.permute(1, 2, 0).cpu().numpy()
    gt_mask_display = gt_mask_tensor.cpu().numpy()

    with torch.no_grad():
        pred = model(image_tensor.unsqueeze(0).to(device))
        pred_mask_display = pred.argmax(dim=1).squeeze().cpu().numpy()

    fig, ax = plt.subplots(1, 3, figsize=(18, 6))

    ax[0].imshow(image_display)
    ax[0].set_title("Original Image")
    ax[0].axis("off")

    ax[1].imshow(gt_mask_display, cmap="tab10")
    ax[1].set_title("Ground Truth Mask")
    ax[1].axis("off")

    ax[2].imshow(pred_mask_display, cmap="tab10")
    ax[2].set_title("Predicted Mask")
    ax[2].axis("off")

    plt.show()

In [ ]:
def overlay_mask(image, mask, alpha=0.5):
    colors = np.array([
        [0, 0, 0],       # background
        [255, 0, 0],     # class 1
        [0, 255, 0],     # class 2
    ])

    overlay = colors[mask]
    return cv2.addWeighted(image, 1-alpha, overlay, alpha, 0)

def visualize_overlay(idx):
    image_tensor, gt_mask_tensor = val_ds[idx]
    image_display = image_tensor.permute(1, 2, 0).cpu().numpy().astype(np.uint8)
    gt_mask_display = gt_mask_tensor.cpu().numpy()

    with torch.no_grad():
        pred = model(image_tensor.unsqueeze(0).to(device))
        pred_mask_display = pred.argmax(dim=1).squeeze().cpu().numpy()

    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    ax[0].imshow(overlay_mask(image_display, gt_mask_display))
    ax[0].set_title("GT Overlay")
    ax[1].imshow(overlay_mask(image_display, pred_mask_display))
    ax[1].set_title("Prediction Overlay")
    ax[0].axis("off")
    ax[1].axis("off")
    plt.show()

In [ ]:
def evaluate_iou(loader):
    model.eval()
    iou = 0

    with torch.no_grad():
        for images, masks in loader:
            images, masks = images.to(device), masks.to(device)
            preds = model(images)
            iou += mean_iou(preds, masks)

    return iou / len(loader)

val_iou = evaluate_iou(val_loader)
print(f"Mean Validation IoU: {val_iou:.4f}")


In [ ]:
dice_metric = smp.losses.DiceLoss(mode="multiclass")

def evaluate_dice(loader):
    dice = 0
    with torch.no_grad():
        for images, masks in loader:
            images, masks = images.to(device), masks.to(device)
            preds = model(images)
            dice += 1 - dice_metric(preds, masks)
    return dice / len(loader)

val_dice = evaluate_dice(val_loader)
print(f"Mean Validation Dice: {val_dice:.4f}")


In [ ]:
def per_class_iou(loader, num_classes=3):
    ious = torch.zeros(num_classes, device=device)

    with torch.no_grad():
        for images, masks in loader:
            images, masks = images.to(device), masks.to(device)
            preds = model(images).argmax(dim=1)

            for cls in range(num_classes):
                inter = ((preds == cls) & (masks == cls)).sum().float()
                union = ((preds == cls) | (masks == cls)).sum().float()
                if union > 0:
                    ious[cls] += inter / union

    return (ious / len(loader)).cpu()

class_iou = per_class_iou(val_loader)
for i, v in enumerate(class_iou):
    print(f"Class {i} IoU: {v:.4f}")


In [ ]:
visualize_prediction(idx=10)

In [ ]:
visualize_prediction(idx=7)
visualize_prediction(idx=11)
visualize_prediction(idx=16)
visualize_prediction(idx=18)
visualize_prediction(idx=30)

In [ ]:
import torch
import segmentation_models_pytorch as smp

device = "cuda" if torch.cuda.is_available() else "cpu"

model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights=None,  # IMPORTANT for inference
    in_channels=3,
    classes=3
).to(device)

model.load_state_dict(
    torch.load("/content/drive/MyDrive/best_unet_week3_fixed.pth",
               map_location=device)
)

model.eval()
print("✅ Model loaded")


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

inference_transform = A.Compose([
    A.Resize(256, 256),  # same as training size
    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    ),
    ToTensorV2()
])


In [ ]:
import cv2
import numpy as np

def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    transformed = inference_transform(image=image)
    image_tensor = transformed["image"].unsqueeze(0)  # (1, C, H, W)

    return image, image_tensor.to(device)


In [ ]:
def predict_mask(model, image_tensor):
    with torch.no_grad():
        output = model(image_tensor)
        prediction = torch.argmax(output, dim=1)  # (1, H, W)
    return prediction.squeeze(0).cpu().numpy()


In [ ]:
import matplotlib.pyplot as plt

def visualize_prediction(original_image, predicted_mask):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.title("Original Image")
    plt.imshow(original_image)
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title("Predicted Mask")
    plt.imshow(predicted_mask, cmap="viridis")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title("Overlay")
    plt.imshow(original_image)
    plt.imshow(predicted_mask, alpha=0.5, cmap="jet")
    plt.axis("off")

    plt.show()


In [ ]:
image_path = "/content/ruin10.png"  # <-- upload your image here

original_image, image_tensor = load_image(image_path)
predicted_mask = predict_mask(model, image_tensor)

visualize_prediction(original_image, predicted_mask)
